<a href="https://colab.research.google.com/github/solomontessema/Generative-AI-with-Python/blob/main/notebooks/Vector_Databases_with_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [4]:
!pip install vec2text

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.0/221.0 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 28.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=05041c48078e37b588f0a0200a7f97d854b3ec5e3c0a204006bdbae46d2f517a
  Store

In [5]:

import pinecone
import openai
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Create Pinecone client
pc = Pinecone(api_key=api_key)

# Create index if it doesn't exist
if "my-index" not in pc.list_indexes().names():
    pc.create_index(
        name="my-index",
        dimension=1536,  # Match your embedding model output
        metric="cosine",  # Or 'euclidean', 'dotproduct'
        spec=ServerlessSpec(
            cloud="aws",  # Or 'gcp'
            region="us-east-1"
        )
    )

# Connect to index
index = pc.Index("my-index")
client = OpenAI(api_key=openai_api_key)

# Embed text using OpenAI
def embed(text):
    response = client.embeddings.create(
        input=[text],
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

# Upsert vector
index.upsert([
    ("concept1", embed("The theory of relativity and its implications"), {"topic": "physics"}),
    ("concept2", embed("The role of empathy in leadership"), {"topic": "psychology"})
])

# Query
query_vector = embed("My cat is so cute")
results = index.query(vector=query_vector, top_k=5, include_metadata=True)
results


{'matches': [{'id': 'concept1',
              'metadata': {'topic': 'physics'},
              'score': 0.718628049,
              'values': []},
             {'id': 'concept2',
              'metadata': {'topic': 'psychology'},
              'score': 0.712097168,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}

In [6]:
query_vector = embed("The theory of relativity")
results = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True,
    filter={"topic": "physics"}  # Only return vectors tagged with topic 'physics'
)
print(results)


{'matches': [{'id': 'concept1',
              'metadata': {'topic': 'physics'},
              'score': 0.94769305,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}


In [8]:
# Updating an existing vector by ID
index.upsert([
    ("concept1", embed("Updated theory of relativity and its modern implications"), {"topic": "physics"})
])

# Deleting a vector by ID
index.delete(ids=["concept2"])

all_vectors = index.query(
    vector=embed("The theory of relativity"),  # Use a relevant query string here
    top_k=10,
    include_metadata=True
)
print(all_vectors)


{'matches': [{'id': 'concept1',
              'metadata': {'topic': 'physics'},
              'score': 0.922424316,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}


Batch Upserts and Queries Explore how to efficiently upsert and query large batches of vectors, which is essential for scaling your application.

Handling Different Similarity Metrics Experiment with different similarity metrics (cosine, euclidean, dotproduct) and understand how they affect search results.

Integrating Pinecone with Applications Build simple applications or APIs that use Pinecone for semantic search or recommendations, combining it with OpenAI for embedding generation.

Performance and Cost Optimization Learn best practices for index dimension sizing, vector pruning, and query tuning to optimize speed and cost.

In [ ]:
# Batch upsert example
texts = [
    "Quantum mechanics and its applications",
    "The psychology of motivation",
    "Advances in renewable energy technology"
]

vectors_to_upsert = []
for i, text in enumerate(texts, start=3):
    vectors_to_upsert.append((f"concept{i}", embed(text), {"topic": "mixed"}))

index.upsert(vectors_to_upsert)

# Query after batch upsert
query_vector = embed("renewable energy")
results = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)
print(results)